## Imports

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from datetime import datetime

In [2]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

## EMA Workbench Setup

In [3]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(7)

Via the `dike_model` variable, we can access scenarios (uncertainties), policies (levers), and our problem formulation (outcomes).

We want to do a run where we analyse a specific, simple, relevant set of policies across a large variety of scenarios based on the outcomes of interest we have deemed relevant for our Client. We seek to produce many data points (each the output of a single experiment, which is defined by the combination of a specific policy under a specific scenario) and compare them to see how these potential policies perform in a broad set of future scenarios.

## Defining Specific Policies

### General remarks:
1. In this early experimentation, we apply all levers at timestamp 0, so at the beginning of the 200years of the simulation. They can also be applied  after 67 or 133 years.
2. In this early experimentation, we are ignoring the policy lever "EWS_DaysToThreat." As analysts on behalf of the Dike Ring as a region, we are more interested in how infrastructural changes can be implemented to protect the region. We are opposed to evacuation as a policy on its own, because this does nothing to protect our Client's constituents' land. In later analysis, it may be worth testing various evacuation levers and effectively treating them as scenario variables.
3. In this early experimentation, we will only study policies that directly impact our Client's region. More can be added for later analysis of policies that the Rijkswaterstaat seems to prefer.

In [4]:
def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

policies = [
    Policy(
        "policy 1",
        # Big RfR in most upstream location (based on the map in ring 1 (non-organic farming)) -
        # our client would prefer this, as they don't care about non-organic farming and it should 
        # significantly decrease flooding of ring 4 (town Gorssel).
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1}
            # **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5} - 
            # this was original "policy 1" - since the course manual states: (for RfR 
            #"Once activated, the project remains active.", I think ""0_RfR 1": 1, "0_RfR 2": 1" 
            # are redundant - but we should evaluate this.
        )
    ),
    Policy(
        "policy 2",
        # Second best alternative of RfR from ring 4 perspective. Some RfR made upstream, 
        # some downstream from them, but none on their area. We choose this to have some comparison for policy 1.
        **dict(
            get_do_nothing_dict(),
            **{"3_RfR 0": 1}
        )
    ),
    Policy(
        "policy 3",
        # Only hightening dikes in ring 4 by the middle height of 0.5m (5 decimeters).
        **dict(
            get_do_nothing_dict(),
            **{"A.4_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 4",
        # Only hightening dikes in ring 4 by maximum height of 1m for comparison with policy 3.
        **dict(
            get_do_nothing_dict(),
            **{"A.4_DikeIncrease 0": 10}
        )
    ),
    Policy(
        "policy 5",
        # policy 1 + policy 3 - trade-off between costs of projects and max safety. Note that making more RfR 
        # (outside of ring 4 though, as they don't want to give up land) 
        # and hightening the dikes in ring 4 by more than 0.5m would increase ring 4 safety even further, but
        # is unrealistic with limiteed budget and stakeholders' interests.
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "A.4_DikeIncrease 0": 5}
        )
    ),
]

## Running Experiments

In [5]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 200
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 200 scenarios * 5 policies * 1 model(s) = 1000 experiments
100%|██████████████████████████████████████| 1000/1000 [03:59<00:00,  4.18it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


## Capturing Results

Detailed analysis of the results will be done in a separate notebook, so this one can exclusively be used for generating experimental results (since this will be an iterative process). An initial view of the results tables is shown below.

In [6]:
CURRENT_DATE = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')

In [7]:
experiments, outcomes = results

In [8]:
experiments_df = pd.DataFrame(experiments)
experiments_df

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,105,84.740803,10.0,0.341607,330.915320,1.0,0.081437,99.946601,1.5,0.198574,...,0,0,0,0,0,0,0,0,policy 1,dikesnet
1,60,317.408704,10.0,0.357019,70.974902,1.5,0.383618,156.311722,10.0,0.480630,...,0,0,0,0,0,0,0,1,policy 1,dikesnet
2,59,41.492248,10.0,0.482082,283.990825,10.0,0.731441,159.979313,1.5,0.979781,...,0,0,0,0,0,0,0,2,policy 1,dikesnet
3,36,76.287353,1.0,0.507364,157.006681,1.5,0.909570,181.330986,1.5,0.579988,...,0,0,0,0,0,0,0,3,policy 1,dikesnet
4,121,189.974602,1.0,0.530336,246.743839,10.0,0.518999,330.866953,1.5,0.757722,...,0,0,0,0,0,0,0,4,policy 1,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,119,266.446552,1.0,0.427957,230.072437,10.0,0.375443,76.988186,1.5,0.341950,...,0,5,0,0,0,0,0,195,policy 5,dikesnet
996,101,232.054999,1.5,0.666962,52.574098,1.0,0.397327,347.407978,10.0,0.029007,...,0,5,0,0,0,0,0,196,policy 5,dikesnet
997,61,246.406567,1.0,0.795631,220.687353,10.0,0.762604,66.889371,1.5,0.959482,...,0,5,0,0,0,0,0,197,policy 5,dikesnet
998,71,299.773054,10.0,0.280876,303.279067,1.5,0.201113,229.159513,10.0,0.262935,...,0,5,0,0,0,0,0,198,policy 5,dikesnet


In [9]:
experiments_df.to_csv('./output/raw/open_exploration/' + CURRENT_DATE + '_experiments.csv')

In [10]:
# TODO: make this work with problem formulation 6
outcomes_df = pd.DataFrame(outcomes)
outcomes_df

,A.1 Expected Annual Damage,A.1 Expected Number of Deaths,A.2 Expected Annual Damage,A.2 Expected Number of Deaths,A.4 Expected Annual Damage,A.4 Expected Number of Deaths,A.5 Expected Annual Damage,A.5 Expected Number of Deaths,Total Infrastructure Costs
0,2.328738e+07,0.018189,6.817814e+08,0.701015,2.812165e+06,0.001312,0.000000e+00,0.000000,8.460000e+07
1,1.921012e+07,0.014007,1.282022e+08,0.131224,0.000000e+00,0.000000,0.000000e+00,0.000000,8.460000e+07
2,0.000000e+00,0.000000,3.738095e+07,0.029584,1.391787e+07,0.006031,2.438747e+08,0.179558,8.460000e+07
3,0.000000e+00,0.000000,6.387116e+06,0.008742,1.844307e+07,0.012206,0.000000e+00,0.000000,8.460000e+07
4,0.000000e+00,0.000000,7.668333e+07,0.071944,4.961275e+07,0.022665,0.000000e+00,0.000000,8.460000e+07
...,...,...,...,...,...,...,...,...,...
995,3.777971e+06,0.002870,1.253114e+08,0.131248,0.000000e+00,0.000000,0.000000e+00,0.000000,9.566877e+07
996,0.000000e+00,0.000000,1.286192e+08,0.118486,0.000000e+00,0.000000,0.000000e+00,0.000000,9.566877e+07
997,0.000000e+00,0.000000,2.729749e+07,0.023549,0.000000e+00,0.000000,0.000000e+00,0.000000,9.566877e+07
998,7.886415e+07,0.041009,4.201030e+08,0.306517,0.000000e+00,0.000000,0.000000e+00,0.000000,9.566877e+07


In [11]:
outcomes_df.to_csv('./output/raw/open_exploration/' + CURRENT_DATE + '_outcomes.csv')